In [ ]:
import sys
import random
import warnings
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import random
import time
import shutil
import glob
!pip install rawpy
!pip install pydicom
!pip install skimage
import rawpy
import imageio
import glob
import pydicom as dicom
%matplotlib inline

from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale, iradon
from tqdm import tqdm
from IPython import display
from sklearn.utils import shuffle
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, ReLU
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# This part will make sure to use a gpu instead of a cpu
tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Nova seção

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def buildModel(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):

    # Build U-Net model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)

    # Block 1
    c1 = Conv2D(2, (2, 2), activation="relu", kernel_initializer="he_normal", padding="same") (s)

    # Block 2
    c2 = Conv2D(4, (2, 2), kernel_initializer="he_normal", padding="same") (c1)
    c2 = BatchNormalization()(c2)
    c2 = LeakyReLU()(c2)

    # Block 3
    c3 = Conv2D(8, (2, 2), kernel_initializer="he_normal", padding="same") (c2)
    c3 = BatchNormalization()(c3)
    c3 = LeakyReLU()(c3)

    # Block 4
    c4 = Conv2D(16, (2, 2), kernel_initializer="he_normal", padding="same") (c3)
    c4 = BatchNormalization()(c4)
    c4 = LeakyReLU()(c4)

    # Block 5
    c5 = Conv2D(16, (2, 2), kernel_initializer="he_normal", padding="same") (c4)
    c5 = BatchNormalization()(c5)
    c5 = LeakyReLU()(c5)

    # Block 6
    c6 = Conv2D(16, (2, 2), kernel_initializer="he_normal", padding="same") (c5)
    c6 = BatchNormalization()(c6)
    c6 = LeakyReLU()(c6)

    # Block 7
    u7 = Conv2DTranspose(32, (2, 2), padding="same")(c6)
    u7 = concatenate([u7, c5])
    c7 = BatchNormalization()(u7)
    c7 = ReLU()(c7)

    # Block 8
    u8 = Conv2DTranspose(32, (2, 2), padding="same")(c7)
    u8 = concatenate([u8, c4])
    c8 = BatchNormalization()(u8)
    c8 = ReLU()(c8)

    # Block 9
    u9 = Conv2DTranspose(16, (2, 2), padding="same")(c8)
    u9 = concatenate([u9, c3])
    c9 = BatchNormalization()(u9)
    c9 = ReLU()(c9)

    # Block 10
    u10 = Conv2DTranspose(8, (2, 2), padding="same")(c9)
    u10 = concatenate([u10, c2])
    c10 = BatchNormalization()(u10)
    c10 = ReLU()(c10)

    # Block 11
    u11 = Conv2DTranspose(4, (2, 2), padding="same")(c10)
    u11 = concatenate([u11, c1])
    #c11 = Dropout(0.1)(u11)
    outputs = Dropout(0.1)(u11)

    #outputs = Conv2D(1, (1, 1), activation="sigmoid") (c11)

    model = Model(inputs = [inputs], outputs = [outputs])
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])
    model.summary()

    return model

In [ ]:
test =  r"/content/drive/MyDrive/DadosAmanda2/"

print(os.listdir(test))

TRAIN_PATH = r"/content/drive/MyDrive/DadosAmanda2/Train"
TEST_PATH = r"/content/drive/MyDrive/DadosAmanda2/Test"
FINAL_TEST_PATH = r"/content/drive/MyDrive/DadosAmanda2/Validation"
dir_path = ""

warnings.filterwarnings("ignore", category = UserWarning, module = "skimage")
seed = 42
random.seed = seed
np.random.seed = seed

"""IMG_HEIGHT = 3023
IMG_WIDTH = 6577
IMG_CHANNELS = 1"""

IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 1 # Esse é o funcional

print(np.version.version)

In [ ]:
# Get train and test IDs
from skimage.transform import resize

def allDivs(n):

    print(n)
    
    div = []

    for i in range(1, n):
        
        if((n % i) == 0):

            div.append(i)
            print("Possível dimensão da imagem : {} x {}".format(i, n/i))

    return div

def getPath(path):

    filename_list = []
    
    for filename in glob.glob(path + "/*.raw"): #assuming all files are raw
    
        filename_list.append(filename)
        
    filename_list = sorted(filename_list)
    
    return filename_list

# 0 = Train
# 1 = Test
# Otherwise this function will return -1 as an error
def getImgs(filename_list, mode = 0):
    
    i = 0

    if(mode == 0):

        image_list = np.zeros((len(filename_list), IMG_HEIGHT, IMG_WIDTH), dtype = np.uint64)
    
    if(mode == 1):
        
        image_list = np.zeros((len(filename_list), IMG_HEIGHT, IMG_WIDTH, 1), dtype = np.uint64)
        
    elif(mode != 0 and mode != 1):
        
        return -1
        
    total = len(filename_list)
    j = 1;

    image_recon = np.zeros((len(filename_list), IMG_HEIGHT, IMG_WIDTH), dtype = np.uint64)

    for x in filename_list:

          print("arquivo {} de {} com nome {}".format(j, total, x))
          rgb = np.fromfile(x, dtype=np.uint64, sep="") # Preciso verificar também aqui os diferentes valores de np.uintX -> melhor : 64|| float.32 também é uma boa
          rgb = rgb/np.mean(rgb)
          print("Dimension of the old image array: ", rgb.ndim)
          print("Size of the old image array: ", rgb.size)
          rgb.resize((IMG_HEIGHT, IMG_WIDTH)) # Esse comando me salvou aqui-> preciso mudar algumas características do sinograma para verificar se funciona aqui
          print("----------------------------------------------------")
          print("The shape of the original image array is: ", rgb.shape)
          print("----------------------------------------------------")
          print("New dimension of the array:", rgb.ndim)
          print(rgb.shape)
          sino, theta = createSin(rgb)
          image_recon[i] = reconstructSin(sino, theta)

          if(mode == 1):

              sino = np.expand_dims(resize(sino, (IMG_HEIGHT, IMG_WIDTH), mode = "constant", preserve_range=True), axis = -1)
              print("Sino.shape : {}".format(sino.shape))

          image_list[i] = sino[:, :]
          j += 1
          i += 1
          
    return image_list, image_recon

def createSin(image, flag=1):

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

  ax1.set_title("Original")
  ax1.imshow(image, cmap=plt.cm.Greys_r)

  print("Creating the sinogram")
  theta = np.linspace(0., 180., max(image.shape), endpoint=False)
  sinogram = radon(image, theta=theta)
  dx, dy = 0.5 * 180.0 / max(image.shape), 0.5 / sinogram.shape[0]
  ax2.set_title("Radon transform\n(Sinogram)")
  ax2.set_xlabel("Projection angle (deg)")
  ax2.set_ylabel("Projection position (pixels)")
  print("Ploting the sinogram")
  ax2.imshow(sinogram, cmap=plt.cm.Greys_r,
            extent=(-dx, 180.0 + dx, -dy, sinogram.shape[0] + dy),
            aspect='auto')

  fig.tight_layout()
  plt.show()

  return sinogram, theta

def reconstructSin(sinogram, theta):
  reconstruction_fbp = iradon(sinogram, theta=theta, filter_name='ramp')

  """plt.title("Reconstruction FBP")
  plt.imshow(reconstruction_fbp, cmap=plt.cm.gray)
  plt.axis("off")
  plt.plot()"""

  return reconstruction_fbp

train_ids = getPath(TRAIN_PATH)
test_ids = getPath(TEST_PATH)
final_test_ids = getPath(FINAL_TEST_PATH)

print("train_ids : {}".format(train_ids))
print("test_ids : {}".format(test_ids))
print("final_test_ids : {}".format(final_test_ids))

In [ ]:
print(final_test_ids[0].split("/")[-1])

In [ ]:
import pathlib
print(pathlib.Path().resolve())

# como base https://github.com/letmaik/rawpy/issues/57#
# Get and resize train images and masks
("\n------ Conjunto de entrada para o treinamento ------\n")
X_train, recon_X_train = getImgs(train_ids, mode = 0)

print("\n------ Conjunto de resposta para o treinamento ------\n")
Y_train, _ = getImgs(test_ids, mode = 1)

print("\n------ Conjunto de entrada para a validação ------\n")
X_test, recon_X_test = getImgs(final_test_ids, mode = 0)

In [ ]:
# Fit model
model = buildModel(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
earlystopper = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint("model_unet_checkpoint.h5", verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=1, epochs=50,
                    callbacks=[checkpointer])
model.save("modelTest.h5")

In [ ]:
def plot_learning_curve(history):

    plt.figure(figsize = (12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])
    plt.title("acurácia do modelo", fontsize = 20)
    plt.ylabel("acurácia (em %)", fontsize = 16)
    plt.xlabel("época", fontsize = 16)
    plt.legend(["treino", "teste"], loc = "upper left")
    # summarize history for loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("perda do modelo", fontsize = 20)
    plt.ylabel("valor perda", fontsize = 16)
    plt.xlabel("época", fontsize = 16)
    plt.legend(["treino", "teste"], loc = "upper left")
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/GráficoTreinamento.png")

In [ ]:
plot_learning_curve(results)
plt.show()

In [ ]:
#Predict on train, val and test
#model = load_model("model.h5")
trainLen = len(X_train)
preds_train = model.predict(X_train[:trainLen - 2], verbose = 1)
preds_test = model.predict(X_train[trainLen - 2:], verbose = 1)
preds_val = model.predict(X_test, verbose = 1)

# Threshold predictions
preds_train_t = (preds_train > 0.25).astype(np.uint8)
preds_test_t = (preds_test > 0.25).astype(np.uint8)
preds_val_t = (preds_val > 0.25).astype(np.uint8)

In [ ]:
#Treino

lastIndex = 0

print(preds_train.shape)
tam_ = len(preds_train)

for i in range(0, len(preds_train)):

    print("Arquivo {} de {}".format(i + 1, tam_))
    plt.figure(figsize = (12, 12))
    
    plt.subplot(1, 3, 1)
    plt.title("treinamento entrada {}".format(train_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(X_train[i], cmap = plt.cm.gray)
  
    plt.subplot(1, 3, 2)
    plt.title("treinamento resposta {}".format(test_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(Y_train[i][:,:, 0], cmap = plt.cm.gray)

    plt.subplot(1, 3, 3)
    plt.title("Resultado Gerado {}".format(train_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(preds_train[i][:, :, 0], cmap = plt.cm.gray)
    
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/ResultadosImagens/Treino/FigTreino{}.png".format(train_ids[i].split("/")[-1]))
    plt.show()

    plt.figure(figsize = (12, 12))

    plt.subplot(1, 3, 1)
    plt.title("Recon dado {}".format(train_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(recon_X_train[i], cmap = plt.cm.gray)

    plt.subplot(1, 3, 2)
    plt.title("Recon saída dado {}".format(test_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(reconstructSin(Y_train[i][:,:, 0], theta = np.linspace(0., 180., max(Y_train[i][:,:, 0].shape), endpoint=False)), cmap = plt.cm.gray)

    plt.subplot(1, 3, 3)
    plt.title("Recon modelo dado {}".format(train_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(reconstructSin(preds_train[i][:, :, 0], theta = np.linspace(0., 180., max(preds_train[i][:, :, 0].shape), endpoint=False)), cmap = plt.cm.gray)

    plt.show()
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/ResultadosImagens/Treino/FigReconTreino{}.png".format(train_ids[i].split("/")[-1]))

    lastIndex = i

In [ ]:
# Teste

tam = len(preds_test)

for i in range(0, tam):

    print("Arquivo {} de {}".format(i + 1, tam))
    plt.figure(figsize = (12, 6))

    plt.subplot(1, 3, 1)
    plt.title("entrada teste {}".format(train_ids[lastIndex + i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(np.squeeze(X_train[lastIndex + i + 1]), cmap = plt.cm.gray)

    plt.subplot(1, 3, 2)
    plt.title("resposta teste {}".format(test_ids[lastIndex + i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(Y_train[lastIndex + i + 1][:, :, 0], cmap = plt.cm.gray)

    plt.subplot(1, 3, 3)
    plt.title("Resultado Gerado teste {}".format(train_ids[lastIndex + i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(preds_test[i][:, :, 0], cmap = plt.cm.gray)
    
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/ResultadosImagens/Teste/FigTest{}.png".format(train_ids[lastIndex + i].split("/")[-1]))
    plt.show()

    plt.figure(figsize = (12, 12))

    plt.subplot(1, 3, 1)
    plt.title("Recon entrada dado {}".format(train_ids[lastIndex + i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(recon_X_train[lastIndex + i], cmap = plt.cm.gray)

    plt.subplot(1, 3, 2)
    plt.title("Recon saída dado {}".format(test_ids[lastIndex + i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(reconstructSin(Y_train[lastIndex + i][:,:, 0], theta = np.linspace(0., 180., max(Y_train[lastIndex + i][:,:, 0].shape), endpoint=False)), cmap = plt.cm.gray)

    plt.subplot(1, 3, 3)
    plt.title("Recon gerada dado {}".format(test_ids[lastIndex + i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(reconstructSin(preds_test[i][:, :, 0], theta = np.linspace(0., 180., max(preds_test[i][:, :, 0].shape), endpoint=False)), cmap = plt.cm.gray)

    plt.show()
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/ResultadosImagens/Teste/FigReconTest{}.png".format(test_ids[lastIndex + i].split("/")[-1]))

In [ ]:
# Validação
tam = len(preds_val)

for i in range(0, tam):

    print("Arquivo {} de {}".format(i + 1, tam))
    plt.figure(figsize = (12, 6))
    
    plt.subplot(1, 2, 1)
    plt.title("entrada validação {}".format(final_test_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(X_test[i], cmap = plt.cm.gray)

    plt.subplot(1, 2, 2)
    plt.axis("off")
    plt.title("Resultado Gerado pela validação {}".format(final_test_ids[i].split("/")[-1]))
    plt.imshow(preds_val[i][:, :, 0], cmap = plt.cm.gray)
    
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/ResultadosImagens/Validacao/FigValid{}.png".format(final_test_ids[i].split("/")[-1]))
    plt.show()

    plt.figure(figsize = (12, 12))

    plt.subplot(1, 2, 1)
    plt.title("Recon entrada do dado {}".format(final_test_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(recon_X_test[i], cmap = plt.cm.gray)

    plt.subplot(1, 2, 2)
    plt.title("Recon gerada pelo modelo do dado {}".format(final_test_ids[i].split("/")[-1]))
    plt.axis("off")
    plt.imshow(reconstructSin(preds_val[i][:, :, 0], theta = np.linspace(0., 180., max(preds_val[i][:, :, 0].shape), endpoint=False)), cmap = plt.cm.gray)

    plt.show()
    plt.savefig(r"/content/drive/MyDrive/Colab Notebooks/ResultadosImagens/Validacao/FigReconTest{}.png".format(final_test_ids[i].split("/")[-1]))